In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import torch

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from tcn import TemporalConvNet
import time
import os


In [2]:
from tcnae import encoder,decoder

In [3]:
import data

In [4]:
data_dir = os.environ['MYDATA_DIR']

In [5]:
data_c = data.Data(wd=data_dir)

In [6]:
raw_x, raw_y = data_c.data(class_num=19, data_num=400)

In [7]:
raw_y.shape

(7600,)

In [8]:
raw_x.shape

(7600, 125, 3)

In [9]:
batch_size = 50
seq_len = 125
epochs = 50
iters = 100
T = 130
n_steps = T + (2 * seq_len)
n_classes = 10  # Digits 0 - 9
n_train = 10000
n_test = 1000

In [10]:
X_train, X_test, y_train, y_test = train_test_split(raw_x, raw_y, test_size=0.90, random_state=0)
X_train.shape

(760, 125, 3)

In [10]:
X_train_new = X_train
y_train_new = y_train
for _ in range(3):
    for i,j in zip(X_train,y_train):
        X_train_new = np.append(X_train_new,i+np.random.normal(0, 1, (125, 3)).reshape(1,125,3))
        y_train_new = np.append(y_train_new, j)
X_train = X_train_new.reshape(-1,125,3)
y_train = y_train_new

In [11]:
num_chans = [6] * (4) 
encoder = encoder(num_inputs=raw_x.shape[1],num_outputs=19, num_channels=num_chans, kernel_size=4, dropout=0.0).cuda()

In [12]:
decoder = decoder(num_inputs=19,num_outputs=3, num_channels=num_chans, kernel_size=4, dropout=0.0).cuda()

In [13]:
criterion = nn.MSELoss().cuda()
lr = 0.001
optimizer1 =optim.RMSprop(encoder.parameters(),lr=lr)
optimizer2 =optim.RMSprop(decoder.parameters(),lr=lr)

In [14]:
encoder.train()
decoder.train()
total_loss = 0
start_time = time.time()
correct = 0
counter = 0

In [15]:
optimizer1

RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.001
    momentum: 0
    weight_decay: 0
)

In [30]:
input = torch.tensor(X_train,dtype=torch.float32).cuda()
for i in range(1000):
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    z = encoder(input)
    out = decoder(z)
    loss = criterion(out, input)
    if i%50==0:
#         input = torch.tensor([X_train[1]],dtype=torch.float32)
#         out= model(input)
#         print(out)
#         print(torch.max(out, 1))
        print(i,loss)
    loss.backward()
    optimizer1.step()
    optimizer2.step()

/mnt/nfs/mizuno/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


0 tensor(8.3974, device='cuda:0', grad_fn=<MseLossBackward>)
50 tensor(7.8561, device='cuda:0', grad_fn=<MseLossBackward>)
100 tensor(8.1499, device='cuda:0', grad_fn=<MseLossBackward>)
150 tensor(7.9120, device='cuda:0', grad_fn=<MseLossBackward>)
200 tensor(7.8734, device='cuda:0', grad_fn=<MseLossBackward>)
250 tensor(7.3472, device='cuda:0', grad_fn=<MseLossBackward>)
300 tensor(7.7668, device='cuda:0', grad_fn=<MseLossBackward>)
350 tensor(7.3444, device='cuda:0', grad_fn=<MseLossBackward>)
400 tensor(7.1886, device='cuda:0', grad_fn=<MseLossBackward>)
450 tensor(7.0865, device='cuda:0', grad_fn=<MseLossBackward>)
500 tensor(7.1314, device='cuda:0', grad_fn=<MseLossBackward>)
550 tensor(7.2292, device='cuda:0', grad_fn=<MseLossBackward>)
600 tensor(6.9728, device='cuda:0', grad_fn=<MseLossBackward>)
650 tensor(7.0028, device='cuda:0', grad_fn=<MseLossBackward>)
700 tensor(6.8877, device='cuda:0', grad_fn=<MseLossBackward>)
750 tensor(6.7983, device='cuda:0', grad_fn=<MseLossBackwa

In [27]:
input = torch.tensor([X_train[10]],dtype=torch.float32).cuda()


In [28]:
z = encoder(input)
z

tensor([[[ 0.2339, -0.8865,  0.3796,  0.9145,  0.1350,  1.2344,  0.5428,
           0.4001,  0.4521, -0.6557,  0.5656,  0.0248, -0.8048, -0.5961,
           1.3202, -0.5260, -1.1610, -0.0127,  0.2765],
         [ 0.3507, -1.0916,  0.4210,  0.6326,  0.2952,  1.3983,  0.0265,
           0.2449,  0.1515, -0.3909,  0.4433, -0.0741, -0.7639, -0.5203,
           1.0080, -0.7416, -1.0706, -0.1304,  0.1544],
         [ 0.3901, -1.0874, -0.3229,  0.2363, -0.3794, -0.7287,  1.9323,
           0.0534,  0.6404, -0.2285, -0.6395, -0.1902, -0.8347, -0.7176,
           1.2532, -0.8445, -1.9996, -1.1653,  1.1234],
         [-0.1248, -0.5096, -0.3915,  0.1537,  0.3825,  1.6372, -0.9823,
          -0.6250, -1.7220,  0.3134,  0.6341,  0.6911, -0.0162, -1.8180,
           0.6760, -1.8018, -0.5127,  0.0340,  0.0374],
         [ 3.1910,  0.1313, -0.7309,  0.3182,  1.0369, -0.5445, -2.0117,
          -0.3426, -1.0849,  0.7778, -1.5263,  1.2675, -1.6433,  2.0858,
           0.1038, -2.5157, -0.9844,  2.9930, 

In [29]:
z

tensor([[[ 0.2339, -0.8865,  0.3796,  0.9145,  0.1350,  1.2344,  0.5428,
           0.4001,  0.4521, -0.6557,  0.5656,  0.0248, -0.8048, -0.5961,
           1.3202, -0.5260, -1.1610, -0.0127,  0.2765],
         [ 0.3507, -1.0916,  0.4210,  0.6326,  0.2952,  1.3983,  0.0265,
           0.2449,  0.1515, -0.3909,  0.4433, -0.0741, -0.7639, -0.5203,
           1.0080, -0.7416, -1.0706, -0.1304,  0.1544],
         [ 0.3901, -1.0874, -0.3229,  0.2363, -0.3794, -0.7287,  1.9323,
           0.0534,  0.6404, -0.2285, -0.6395, -0.1902, -0.8347, -0.7176,
           1.2532, -0.8445, -1.9996, -1.1653,  1.1234],
         [-0.1248, -0.5096, -0.3915,  0.1537,  0.3825,  1.6372, -0.9823,
          -0.6250, -1.7220,  0.3134,  0.6341,  0.6911, -0.0162, -1.8180,
           0.6760, -1.8018, -0.5127,  0.0340,  0.0374],
         [ 3.1910,  0.1313, -0.7309,  0.3182,  1.0369, -0.5445, -2.0117,
          -0.3426, -1.0849,  0.7778, -1.5263,  1.2675, -1.6433,  2.0858,
           0.1038, -2.5157, -0.9844,  2.9930, 